In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('new_data/complete-data.csv')

data.sort_values('year')

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
308,191,2,2,569,13.0,0.38,0.85,0.0,0.00,0.00,...,180,0.0,3,1.5,1.0,9.90,-86.415,0.0,24.90,5.02
736,393,2,0,567,28.0,1.18,1.96,0.0,7.67,0.33,...,174,0.0,3,0.0,0.0,5.04,-44.760,0.0,28.00,6.04
230,136,2,1,573,10.0,0.50,1.40,0.1,0.00,0.00,...,160,0.0,3,2.0,1.0,11.52,-181.040,0.0,27.75,8.52
225,133,2,0,563,3.0,0.00,0.67,0.0,0.00,0.00,...,150,0.0,3,1.0,0.5,13.68,-68.565,0.0,14.85,5.68
121,79,3,0,570,28.0,0.86,1.61,0.0,0.00,0.00,...,162,0.0,3,0.0,0.0,11.70,-189.260,0.0,27.55,7.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,475,11,0,568,0.0,0.00,0.00,0.0,0.00,0.00,...,178,0.0,11,1.0,0.5,15.66,-348.260,0.0,65.35,11.05
615,334,11,0,556,0.0,0.00,0.00,0.0,0.00,0.00,...,175,0.0,7,2.5,1.0,6.66,-72.975,0.0,19.65,6.09
630,342,11,0,574,0.0,0.00,0.00,0.0,0.00,0.00,...,164,0.0,6,1.5,0.5,12.96,-31.665,0.0,5.60,3.12
680,372,11,0,568,0.0,0.00,0.00,0.0,0.00,0.00,...,145,0.0,11,0.5,0.5,9.54,-195.050,0.0,40.05,8.69


In [3]:
with open('stats.txt', 'w') as file:
    for col in data.columns:
        if type(data[col][0]) == str: 
            continue
        file.write("-"*50 + "\n")
        file.write(f"Statistics of {col}\n")
        file.write("-"*50 + "\n")
        # data[col].plot.box();  # You can enable this if you want to create a plot

        q3, q1 = np.percentile(data[col], [75 ,25])

        file.write(f"Min: {data[col].min()}\n")
        file.write(f"Max: {data[col].max()}\n")
        file.write(f"Mean: {data[col].mean()}\n")
        file.write(f"Median: {data[col].median()}\n")
        file.write(f"Standard deviation: {data[col].std()}\n")
        file.write(f"Skewness: {data[col].skew()}\n")
        file.write(f"Variance: {data[col].var()}\n")
        file.write(f"Kurtosis: {data[col].kurtosis()}\n")
        file.write(f"Interquartile Range: {q3 - q1} | Q1: {q1} | Q3: {q3}\n")

In [ ]:
outliers_std = {}
all_outliers = []  # Array to store all outlier rows

#columns to skip
not_skip_cols = ['height', 'weight']

for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = pd.concat([data.loc[data[col] < lower], data.loc[data[col] > upper]])
    outliers_std[col] = outliers
    all_outliers.extend(outliers.values.tolist())  # Add outliers to the array

for key in outliers_std.keys():
    if len(outliers_std[key]) > 0:
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_std[key][key])

# Printing all outlier rows
print("All outlier rows:")
for outlier in all_outliers:
    print(outlier)

--------------------------------------------------
Outliers of height - mean: 71.61170212765957 - std: 6.500703333992922
--------------------------------------------------
149    9.0
150    9.0
151    9.0
152    9.0
153    9.0
154    9.0
Name: height, dtype: float64
--------------------------------------------------
Outliers of weight - mean: 167.49601063829786 - std: 21.355278507877653
--------------------------------------------------
225    240
226    240
293    240
294    240
295    240
420    232
421    232
Name: weight, dtype: int64
All outlier rows:
[132.0, 4.0, 0.0, 564.0, 34.0, 1.32, 6.06, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.72, 0.45, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9, 1.0, 11.94, 7.38, 1.71, 0.56, 2.94, -281.47, 15.84, 95.6, 11.36, 27.24, 3.0, 582.0, 0.0, 701.0, 0.0, 9.0, 223.0, 0.0, 4.0, 0.3333333333333333, 0.6666666666666666, 14.28, -244.25, 0.0, 101.73333333333332, 12.54]
[132.0, 5.0, 0.0, 564.0, 34.0, 0.88, 4.06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
outliers_ids_std = set([val for key in outliers_std.keys() for val in outliers_std[key]['playerID']])
outlier_ratio = len(outliers_ids_std) / len(data)
outlier_ratio

0.005319148936170213

In [ ]:
outliers_z = {}
for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    z_val = (data[col] - data[col].mean()) / data[col].std()
    outliers_z[col] = data.loc[(z_val < -4) | (z_val > 4)]

for key in outliers_z.keys():
    if (len(outliers_z[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_z[key][key])

--------------------------------------------------
Outliers of height - mean: 71.61170212765957 - std: 6.500703333992922
--------------------------------------------------
149    9.0
150    9.0
151    9.0
152    9.0
153    9.0
154    9.0
Name: height, dtype: float64


In [ ]:
outliers_ids_z = set([val for key in outliers_z.keys() for val in outliers_z[key]['playerID']])
outlier_ratio = len(outliers_ids_z) / len(data)
outlier_ratio

0.0013297872340425532

In [ ]:
outliers_ids = outliers_ids_std.union(outliers_ids_z)
print(f"Total outliers: {len(outliers_ids)}")
print(f"Total outliers ratio: {len(outliers_ids) / len(data)}")
print(outliers_ids)
#create a new df with outliers removed
manual_outlier_df = data[~data['playerID'].isin(outliers_ids)]

manual_outlier_df.duplicated(subset=['playerID', 'year', 'stint']).sum()

Total outliers: 4
Total outliers ratio: 0.005319148936170213
{132, 245, 326, 199}


3

In [ ]:

manual_outlier_df.to_csv('new_data/complete-data.csv', index=False)
manual_outlier_df.sort_values('year')
# manual_outlier_df.to_csv('output/loan_dev_transformed_without_outliers.csv', index=False)

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,weight,award_count,career_year,playoff_progression_rolling,playoff_rolling,pp36_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,defensive_discipline_rolling
211,191,3,0,568,3.0,0.33,0.33,0.0,0.00,0.00,...,180,0.0,4,1.000000,0.666667,10.08,-67.070000,0.000000,20.433333,4.266667
521,393,3,0,567,31.0,1.32,2.45,0.0,0.00,0.00,...,174,0.0,4,0.333333,0.333333,5.52,-57.413333,0.000000,28.200000,5.906667
332,274,3,1,555,5.0,0.40,2.80,0.2,0.00,0.00,...,143,0.0,4,0.000000,0.000000,18.96,-230.903333,0.333333,47.033333,11.240000
467,366,3,2,560,16.0,1.25,4.56,0.0,0.00,0.00,...,165,0.0,4,0.000000,0.000000,14.88,-193.226667,0.000000,58.133333,10.333333
698,504,3,2,569,11.0,0.64,2.36,0.0,36.67,8.33,...,138,0.0,4,2.000000,1.000000,5.76,-83.516667,0.000000,30.066667,5.353333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,306,11,0,571,0.0,0.00,0.00,0.0,0.00,0.00,...,155,0.0,7,2.666667,0.666667,11.04,-108.586667,0.000000,19.633333,3.273333
750,548,11,0,564,0.0,0.00,0.00,0.0,0.00,0.00,...,165,0.0,5,2.000000,1.000000,19.08,-317.120000,0.000000,59.800000,8.746667
397,319,11,0,555,0.0,0.00,0.00,0.0,0.00,0.00,...,188,0.0,12,2.000000,0.666667,13.56,-144.650000,0.000000,64.366667,9.066667
316,260,11,0,574,0.0,0.00,0.00,0.0,0.00,0.00,...,180,0.0,6,1.000000,0.666667,12.84,-235.453333,0.000000,52.833333,9.020000
